In [65]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import Libraries

In [105]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam

In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

Loading the Dataset

In [6]:
####import data

data = pd.read_csv('test_file.csv')
data.head()  ###activity, ax, ay, az, gx, gy, gz

,activity,ax,ay,az,gx,gy,gz
0,normal,-2.07,4.00,8.22,-0.540,-1.06,0.907
1,normal,-2.13,4.03,8.20,-0.515,-1.08,0.910
2,normal,-2.20,4.10,8.22,-0.491,-1.10,0.913
3,normal,-2.28,4.14,8.23,-0.459,-1.12,0.917
4,normal,-2.36,4.22,8.24,-0.431,-1.13,0.919


In [69]:
# data.shape        for checking purpose... it is alreadt labelled

(26273, 7)

In [12]:
data['activity']

0          normal
1          normal
2          normal
3          normal
4          normal
           ...   
26268    upstairs
26269    upstairs
26270    upstairs
26271    upstairs
26272         NaN
Name: activity, Length: 26273, dtype: object

In [20]:
new_data = data.dropna(axis=0).copy()
new_data.head()

,activity,ax,ay,az,gx,gy,gz,label
0,normal,-2.07,4.00,8.22,-0.540,-1.06,0.907,1
1,normal,-2.13,4.03,8.20,-0.515,-1.08,0.910,1
2,normal,-2.20,4.10,8.22,-0.491,-1.10,0.913,1
3,normal,-2.28,4.14,8.23,-0.459,-1.12,0.917,1
4,normal,-2.36,4.22,8.24,-0.431,-1.13,0.919,1


In [21]:
new_data['label'].value_counts()

1    18031
0     4488
2     3753
Name: label, dtype: int64

In [22]:
new_data['activity'].value_counts()

normal        18031
downstairs     4488
upstairs       3753
Name: activity, dtype: int64

Label the Data Like Normal as 1, Upstairs as 2 and so on

In [23]:
label = LabelEncoder()                       ###labelling the string to integers

new_data['label'] = label.fit_transform(new_data['activity'])
new_data.head()

,activity,ax,ay,az,gx,gy,gz,label
0,normal,-2.07,4.00,8.22,-0.540,-1.06,0.907,1
1,normal,-2.13,4.03,8.20,-0.515,-1.08,0.910,1
2,normal,-2.20,4.10,8.22,-0.491,-1.10,0.913,1
3,normal,-2.28,4.14,8.23,-0.459,-1.12,0.917,1
4,normal,-2.36,4.22,8.24,-0.431,-1.13,0.919,1


In [25]:
new_data['label'].value_counts()

1    18031
0     4488
2     3753
Name: label, dtype: int64

In [26]:
label.classes_   ##checking the label

array(['downstairs', 'normal', 'upstairs'], dtype=object)

In [27]:
new_data['activity'].value_counts()               ###for checking the balance of data

normal        18031
downstairs     4488
upstairs       3753
Name: activity, dtype: int64

In [28]:
new_data['ax'] = new_data['ax'].astype(float)         ###convert the data to float
new_data['ay'] = new_data['ay'].astype(float)
new_data['az'] = new_data['az'].astype(float)
new_data['gx'] = new_data['gx'].astype(float)
new_data['gy'] = new_data['gy'].astype(float)
new_data['gz'] = new_data['gz'].astype(float)

In [ ]:
####plotting the data as per the activity /// if needed


In [29]:
normal = new_data[new_data['activity']=='normal'].head(3753).copy()                 #### not needed in final code as data will be balanced.... 
                                                                            #### This is just a random csv file I made to check the code
downstairs = new_data[new_data['activity']=='downstairs'].head(3753).copy()
upstairs = new_data[new_data['activity']=='upstairs'].head(3753).copy()

In [30]:
final_data = pd.DataFrame()                            ####creating dataframe.... Not needed
final_data = final_data.append([normal, upstairs, downstairs])
final_data.shape
final_data.head()

,activity,ax,ay,az,gx,gy,gz,label
0,normal,-2.07,4.00,8.22,-0.540,-1.06,0.907,1
1,normal,-2.13,4.03,8.20,-0.515,-1.08,0.910,1
2,normal,-2.20,4.10,8.22,-0.491,-1.10,0.913,1
3,normal,-2.28,4.14,8.23,-0.459,-1.12,0.917,1
4,normal,-2.36,4.22,8.24,-0.431,-1.13,0.919,1


In [31]:
final_data.value_counts()
final_data['activity'].value_counts()

normal        3753
upstairs      3753
downstairs    3753
Name: activity, dtype: int64

In [33]:
 ### Data Standarisation

 x = final_data[['ax', 'ay', 'az', 'gx', 'gy', 'gz']]
 y = final_data['label']

In [38]:
scaler = StandardScaler()
X = scaler.fit_transform(x)

scaled_X = pd.DataFrame(data = X, columns=['ax', 'ay', 'az', 'gx', 'gy', 'gz'])
scaled_X['label'] = y.values

scaled_X

,ax,ay,az,gx,gy,gz,label
0,-1.236010,3.193491,1.793126,-0.634609,-1.299356,1.030879,1
1,-1.256316,3.201316,1.785786,-0.602253,-1.324355,1.034480,1
2,-1.280007,3.219573,1.793126,-0.571191,-1.349354,1.038081,1
3,-1.307081,3.230006,1.796796,-0.529775,-1.374353,1.042883,1
4,-1.334156,3.250872,1.800466,-0.493536,-1.386853,1.045283,1
...,...,...,...,...,...,...,...
11254,1.007811,0.001018,-0.243684,0.073515,-0.105651,-0.057218,0
11255,1.051807,-0.001591,-0.258363,0.065081,-0.131900,-0.058754,0
11256,1.156722,-0.035498,-0.258363,0.078007,-0.143149,-0.050277,0
11257,1.116110,-0.051147,-0.295063,0.092114,-0.151899,-0.043591,0


In [42]:
import scipy.stats as stats

In [75]:
###Frame Preparation

Fs = 50

frame_size = Fs*4
hop_size = Fs*2

In [76]:
def get_frames(data, frame_size, hop_size):

    N_FEATURES = 6

    frames = []
    labels = []
    for i in range(0, len(data) - frame_size, hop_size):
        x1 = data['ax'].values[i: i + frame_size]
        y1 = data['ay'].values[i: i + frame_size]
        z1 = data['az'].values[i: i + frame_size]
        x2 = data['gx'].values[i: i + frame_size]
        y2 = data['gy'].values[i: i + frame_size]
        z2 = data['gz'].values[i: i + frame_size]
        
        # Retrieve the most often used label in this segment
        label = stats.mode(data['label'][i: i + frame_size])[0][0]
        frames.append([x1, y1, z1, x2, y2, z2])
        labels.append(label)

    # Bring the segments into a better shape
    frames = np.asarray(frames).reshape(-1, frame_size, N_FEATURES)
    labels = np.asarray(labels)

    return frames, labels

In [97]:
X, y = get_frames(scaled_X, frame_size, hop_size)

X.shape, y.shape

((111, 200, 6), (111,))

In [98]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0, stratify = y)
X_train.shape, X_test.shape

((88, 200, 6), (23, 200, 6))

In [99]:
X_train[0].shape, X_test[0].shape

((200, 6), (200, 6))

In [100]:
X_train = X_train.reshape(88,200,6,1)
X_test = X_test.reshape(23,200,6,1)

In [101]:
X_train[0].shape, X_test[0].shape

((200, 6, 1), (200, 6, 1))

In [102]:
model = Sequential()
model.add(Conv2D(16,(2,2),activation = 'relu', input_shape = X_train[0].shape))
model.add(Dropout(0.1))

model.add(Conv2D(32,(2,2), activation = 'relu'))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(6, activation = 'softmax'))  # 'softmax' Because we have multi class (6) classification problem.

In [107]:
model.compile(optimizer = Adam(learning_rate = 0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [108]:
history = model.fit(X_train, y_train, epochs = 10, validation_data = (X_test, y_test), verbose = 1)

Epoch 1/10
3/3 [==============================] - 1s 109ms/step - loss: 1.6371 - accuracy: 0.3068 - val_loss: 1.2144 - val_accuracy: 0.5652
Epoch 2/10
3/3 [==============================] - 0s 51ms/step - loss: 1.1983 - accuracy: 0.5114 - val_loss: 1.3962 - val_accuracy: 0.4783
Epoch 3/10
3/3 [==============================] - 0s 49ms/step - loss: 0.9183 - accuracy: 0.6477 - val_loss: 1.3521 - val_accuracy: 0.5217
Epoch 4/10
3/3 [==============================] - 0s 49ms/step - loss: 0.7981 - accuracy: 0.6250 - val_loss: 1.3650 - val_accuracy: 0.5217
Epoch 5/10
3/3 [==============================] - 0s 52ms/step - loss: 0.6516 - accuracy: 0.7386 - val_loss: 1.5615 - val_accuracy: 0.5217
Epoch 6/10
3/3 [==============================] - 0s 49ms/step - loss: 0.5274 - accuracy: 0.8182 - val_loss: 1.6828 - val_accuracy: 0.5217
Epoch 7/10
3/3 [==============================] - 0s 50ms/step - loss: 0.6467 - accuracy: 0.7841 - val_loss: 1.7196 - val_accuracy: 0.5652
Epoch 8/10
3/3 [==========